In [1]:
from Modified_lux3_wrapper.modified_wrappers_20250226_01 import LuxAIS3GymEnv
import numpy as np
from Modified_stablebaseline3_PPO.modified_ppo_20250226_01 import PPO
import torch
from gymnasium import spaces

In [2]:
env = LuxAIS3GymEnv(numpy_output=True)

In [3]:
model = PPO("MultiInputPolicy", env, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [4]:
model.policy

MultiInputActorCriticPolicy(
  (features_extractor): CombinedExtractor(
    (extractors): ModuleDict(
      (enemy_energies): Flatten(start_dim=1, end_dim=-1)
      (enemy_positions): Flatten(start_dim=1, end_dim=-1)
      (enemy_spawn_location): Flatten(start_dim=1, end_dim=-1)
      (enemy_visible_mask): Flatten(start_dim=1, end_dim=-1)
      (map_explored_status): Flatten(start_dim=1, end_dim=-1)
      (map_features_energy): Flatten(start_dim=1, end_dim=-1)
      (map_features_tile_type): Flatten(start_dim=1, end_dim=-1)
      (match_steps): Flatten(start_dim=1, end_dim=-1)
      (my_spawn_location): Flatten(start_dim=1, end_dim=-1)
      (relic_nodes): Flatten(start_dim=1, end_dim=-1)
      (relic_nodes_mask): Flatten(start_dim=1, end_dim=-1)
      (sensor_mask): Flatten(start_dim=1, end_dim=-1)
      (steps): Flatten(start_dim=1, end_dim=-1)
      (team_id): Flatten(start_dim=1, end_dim=-1)
      (team_points): Flatten(start_dim=1, end_dim=-1)
      (team_wins): Flatten(start_dim=

In [5]:
model.policy.device

device(type='cuda', index=0)

In [6]:
obs = {
    "enemy_energies": np.random.randint(-800, 401, size=(1, 16,), dtype=np.int32),
    "enemy_positions": np.random.randint(-1, 24, size=(1, 16, 2), dtype=np.int32),
    "enemy_spawn_location": np.random.randint(-1, 24, size=(1, 2,), dtype=np.int32),
    "enemy_visible_mask": np.random.randint(0, 2, size=(1, 16,), dtype=np.int32),
    "map_explored_status": np.random.randint(0, 2, size=(1, 24, 24), dtype=np.int32),
    "map_features_energy": np.random.randint(-7, 10, size=(1, 24, 24), dtype=np.int32),
    "map_features_tile_type": np.random.randint(-1, 3, size=(1, 24, 24), dtype=np.int32),
    "match_steps": np.random.randint(0, 101, size=(1, 1,), dtype=np.int32),
    "my_spawn_location": np.random.randint(-1, 24, size=(1, 2,), dtype=np.int32),
    "relic_nodes": np.random.randint(-1, 24, size=(1, 6, 2), dtype=np.int32),
    "relic_nodes_mask": np.random.randint(0, 2, size=(1, 6,), dtype=np.int32),
    "sensor_mask": np.random.randint(0, 2, size=(1, 24, 24), dtype=np.int32),
    "steps": np.random.randint(0, 506, size=(1, 1,), dtype=np.int32),
    "team_id": np.random.randint(0, 2, size=(1, 1,), dtype=np.int32),
    "team_points": np.random.randint(0, 2501, size=(1, 2,), dtype=np.int32),
    "team_wins": np.random.randint(0, 4, size=(1, 2,), dtype=np.int32),
    "unit_active_mask": np.random.randint(0, 2, size=(1, 16,), dtype=np.int32),
    "unit_energies": np.random.randint(-800, 401, size=(1, 16,), dtype=np.int32),
    "unit_move_cost": np.random.randint(1, 6, size=(1, 1, ), dtype=np.int32),
    "unit_positions": np.random.randint(-1, 24, size=(1, 16, 2), dtype=np.int32),
    "unit_sap_cost": np.random.randint(30, 51, size=(1, 1, ), dtype=np.int32),
    "unit_sap_range": np.random.randint(3, 8, size=(1, 1, ), dtype=np.int32),
    "unit_sensor_range": np.random.randint(2, 5, size=(1, 1, ), dtype=np.int32),
}

In [7]:
obs = {k: torch.tensor(v, dtype=torch.float32, device="cuda") for k, v in obs.items()}

# Convert observation to tensor and check shape
obs_tensor = model.policy.extract_features(obs)
print(f"Extracted Feature Shape: {obs_tensor.shape}")  # Expected: (batch_size, 2464)

Extracted Feature Shape: torch.Size([1, 2466])


In [8]:
with torch.no_grad():
    action_distribution, value, log = model.policy.forward(obs)

In [9]:
action_distribution.shape

torch.Size([1, 48])

In [10]:
action_distribution

tensor([[ 1,  4,  2,  1,  6,  2,  1,  7,  0,  1, 11,  2,  4, 12, 11,  0, 13,  1,
          0, 12, 11,  2, 12, 10,  3, 12,  8,  3,  7,  3,  1, 14,  8,  2, 10,  4,
          1, 14,  7,  3,  9,  2,  0,  9, 12,  0,  5,  6]], device='cuda:0')

In [11]:
value.shape

torch.Size([1, 1])

In [12]:
value

tensor([[-0.8707]], device='cuda:0')

In [13]:
log.shape

torch.Size([1])

In [14]:
log

tensor([-115.3218], device='cuda:0')

In [15]:
actions = action_distribution.reshape(-1, 16, 3)
actions

tensor([[[ 1,  4,  2],
         [ 1,  6,  2],
         [ 1,  7,  0],
         [ 1, 11,  2],
         [ 4, 12, 11],
         [ 0, 13,  1],
         [ 0, 12, 11],
         [ 2, 12, 10],
         [ 3, 12,  8],
         [ 3,  7,  3],
         [ 1, 14,  8],
         [ 2, 10,  4],
         [ 1, 14,  7],
         [ 3,  9,  2],
         [ 0,  9, 12],
         [ 0,  5,  6]]], device='cuda:0')

In [16]:
actions.shape

torch.Size([1, 16, 3])

In [17]:
model.policy.mlp_extractor = torch.compile(model.policy.mlp_extractor)

In [18]:
with torch.no_grad():
    action_distribution, value, log = model.policy.forward(obs)

In [ ]:
action_distribution

In [ ]:
%pip install --upgrade luxai-s3